In [1]:
import pandas as pd

df = pd.read_csv('processed_data/first_pass.csv')

In [2]:
head = 0

In [3]:
df['Time'] = pd.to_datetime(df['Time'], utc=True, format='ISO8601')

# Sort the dataframe by 'Time' within each 'RunID'
df = df.sort_values(by=['RunID', 'Time']).reset_index(drop=True)

# Calculate the time difference and distance difference between each row and the previous row within each 'RunID'
df['Time_diff'] = df.groupby('RunID')['Time'].diff().dt.total_seconds()
df['Distance_diff'] = df.groupby('RunID')['Distance'].diff()

# Calculate the speed in m/s
df['Speed'] = df['Distance_diff'] / df['Time_diff']

# Display the dataframe with calculated speed values
df.head(head)

,RunID,Time,Latitude,Longitude,Elevation,Distance,HeartRate,Cadence,Speed,Time_diff,Distance_diff


In [4]:
def calculate_statistics(group):
    time_range = pd.Timedelta(minutes=5)
    statistics = {}
    
    # Iterate over each minute in the first 5 minutes
    for i in range(1, 6):
        start_time = group['Time'].min() + pd.Timedelta(minutes=i - 1)
        end_time = group['Time'].min() + pd.Timedelta(minutes=i)
        
        # Filter the data for the current minute
        current_minute_data = group[(group['Time'] >= start_time) & (group['Time'] < end_time)]
        
        # Calculate statistics for heart rate, elevation, speed and cadence for the current minute
        statistics[f'HeartRate_mean_minute_{i}'] = current_minute_data['HeartRate'].mean()
        statistics[f'Elevation_mean_minute_{i}'] = current_minute_data['Elevation'].mean()
        statistics[f'Cadence_mean_minute_{i}'] = current_minute_data['Cadence'].mean()
        statistics[f'Speed_mean_minute_{i}'] = current_minute_data['Speed'].mean()
    
    return pd.Series(statistics)



df_statistics = df.groupby('RunID').apply(calculate_statistics, include_groups=False).reset_index()

In [5]:
display(df_statistics.head(head))
df_statistics.to_csv("processed_data/train_data.csv")

,RunID,HeartRate_mean_minute_1,Elevation_mean_minute_1,Cadence_mean_minute_1,Speed_mean_minute_1,HeartRate_mean_minute_2,Elevation_mean_minute_2,Cadence_mean_minute_2,Speed_mean_minute_2,HeartRate_mean_minute_3,...,Cadence_mean_minute_3,Speed_mean_minute_3,HeartRate_mean_minute_4,Elevation_mean_minute_4,Cadence_mean_minute_4,Speed_mean_minute_4,HeartRate_mean_minute_5,Elevation_mean_minute_5,Cadence_mean_minute_5,Speed_mean_minute_5
